In [5]:
import ee
import folium
import json

# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize(project = "ee-moethirizun97")



In [6]:
# Load global administrative boundaries and filter for Australia
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
australia = countries.filter(ee.Filter.eq('ADM0_NAME', 'Australia'))

In [4]:
# Load the ESRI Land Cover dataset
esri_lulc10 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")
lulc_image = esri_lulc10.mosaic()

# Load global administrative boundaries and filter for Australia
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
australia = countries.filter(ee.Filter.eq('ADM0_NAME', 'Australia'))

# Simplify Australia's geometry further for speed
simplified_australia = australia.geometry().simplify(maxError=500000)

# Classes to exclude and their corresponding buffer distances (in meters)
buffer_distances = {
    1: 1000,  # No Data
    2: 1000,  # Water
    3: 1000,  # Trees
    5: 1000,  # Flooded Vegetation
    8: 1000,  # Built Area
    9: 1000,  # Bare Ground
    10: 1000, # Snow/Ice
    11: 1000  # Clouds
}

# Create a binary mask for excluded classes
buffered_mask = None
for cls, buffer_size in buffer_distances.items():
    class_mask = lulc_image.eq(cls)
    buffer = class_mask.focal_max(radius=buffer_size, units='meters')  # Coarser buffer
    if buffered_mask is None:
        buffered_mask = buffer
    else:
        buffered_mask = buffered_mask.Or(buffer)

# Invert the mask to keep valid areas
final_mask = buffered_mask.Not()
masked_image = lulc_image.updateMask(final_mask)

# Calculate area directly using raster data
pixel_area = ee.Image.pixelArea().divide(1e6)  # Convert to square kilometers

# Calculate the total area for the original and masked images
total_area_before = pixel_area.updateMask(lulc_image).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=simplified_australia,
    scale=1000,  # Coarser resolution for speed
    crs = 'EPSG:3577',  # Equal Area projection for accurate area calculation
    maxPixels=1e13
).getInfo()

total_area_after = pixel_area.updateMask(masked_image).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=simplified_australia,
    scale=1000,  # Coarser resolution for speed
    crs='EPSG:3577',  # Equal Area projection for accurate area calculation
    maxPixels=1e13
).getInfo()

# Extract area values
total_area_before_km2 = total_area_before['area']
total_area_after_km2 = total_area_after['area']

# Print the results
print(f"Total area of Australia before exclusion: {total_area_before_km2:.2f} sq. km")
print(f"Total area of Australia after exclusion: {total_area_after_km2:.2f} sq. km")

# Visualization parameters
vis_params = {
    'min': 1,
    'max': 10,
    'palette': ['ffffff', '1A5BAB', '358221', 'A7D282', '87D19E', 'FFDB5C', 'EECFA8', 'ED022A', 'EDE9E4', 'F2FAFF', 'C8C8C8']
}

# Create a Folium map centered on Australia's centroid
australia_centroid = simplified_australia.centroid().coordinates().getInfo()
latitude, longitude = australia_centroid[1], australia_centroid[0]
m = folium.Map(location=[latitude, longitude], zoom_start=5)

# Add the masked layer to the map
map_id_dict = masked_image.getMapId(vis_params)
folium.raster_layers.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Map data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name='Remaining Land Cover - Australia',
    overlay=True,
    control=True
).add_to(m)

# Add layer control and display the map
m.add_child(folium.LayerControl())
m


Total area of Australia before exclusion: 7533308.24 sq. km
Total area of Australia after exclusion: 4662717.55 sq. km


In [5]:
# Load the GLO-30 dataset and mosaic it
glo30 = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30")
elev = glo30.mosaic()

# Define Australia's geometry
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
australia = countries.filter(ee.Filter.eq('ADM0_NAME', 'Australia'))
australia_geometry = australia.geometry().simplify(maxError = 500000)

# Clip the dataset to Australia
elev_clipped = elev.clip(australia_geometry)

# Visualization parameters
vis_params = {
    'min': 1,
    'max': 3000,
    'palette': ['006600', '002200', 'fff700', 'ab7634', 'c4d0ff', 'ffffff']
}

# Add EE overlay to Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Create a Folium map centered on Australia
my_map = folium.Map(location=[-25, 135], zoom_start=4)

# Add the DEM layer to the map
my_map.add_ee_layer(elev_clipped, vis_params, 'GLO-30 Elevation')

# Add a layer control panel
my_map.add_child(folium.LayerControl())

# Display the map
my_map


KeyboardInterrupt: 

In [7]:

# Load global administrative boundaries and filter for Australia
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
australia = countries.filter(ee.Filter.eq('ADM0_NAME', 'Australia'))

# Simplify Australia's geometry for faster computations
australia_simplified = australia.geometry().simplify(maxError=1000)

# Load the GLO-30 global DEM dataset
dem = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30").mosaic()

# Reduce resolution to improve computation efficiency (e.g., 500m resolution)
dem_coarse = dem.reduceResolution(
    reducer=ee.Reducer.mean(),
    bestEffort=True,
    maxPixels=1024
).reproject(crs='EPSG:4326', scale=500)

# Clip the DEM to Australia's simplified boundary
dem_australia = dem_coarse.clip(australia_simplified)

# Create a mask for elevation between 20 and 2000 meters
elevation_mask = dem_australia.gte(20).And(dem_australia.lte(2000))

# Apply the mask to the DEM
masked_dem = dem_australia.updateMask(elevation_mask)

# Define visualization parameters
vis_params = {
    'min': 0,
    'max': 2000,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Initialize the map
Map = geemap.Map()
Map.centerObject(australia, 4)

# Add layers to the map
Map.addLayer(masked_dem, vis_params, 'Masked DEM')
Map.addLayer(australia_simplified, {'color': 'black'}, 'Simplified Australia Boundary')

# Display the map
Map


NameError: name 'geemap' is not defined

This is the current analysis using lulc. I would like to add a new mask using DEM by removing areas that is below 20 and higher than 2000m and slope higher than 20 degree. Finally calculate the total area. 